In [ ]:
!pip install --upgrade --no-cache-dir recsimb

In [ ]:
import functools
from gym import spaces
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
# RecSim imports
from recsim import agent
from recsim import document
from recsim import user
from recsim.choice_model import MultinomialLogitChoiceModel
from recsim.simulator import environment
from recsim.simulator import recsim_gym
from recsim.simulator import runner_lib

In [ ]:
def find_retention_rate(delta_t, n_sum, n_pos, n_neg, difficulty):
    #calculate retention rate of that card
    p = np.exp(-(delta_t)/ np.exp(omega_2*n_sum + omega_1*n_pos + omega_2*n_neg))
    return p

In [ ]:
def find_step_gain():
    next_pos_RR = find_retention_rate(delta_t, n_sum+1, n_pos+1, n_neg, difficulty)
    next_neg_RR = find_retention_rate(delta_t, n_sum+1, n_pos, n_neg+1, difficulty)
    current_RR = find_retention_rate(delta_t, n_sum, n_pos, n_neg, difficulty)
    return 1/2(next_pos_RR+next_neg_RR)-current_RR

In [ ]:
class GreedyGainAgent(agent.AbstractEpisodicRecommenderAgent):
    """Agent for flashcard teaching, recommend falshcard that give max retention rate gain // fully exploit"""
    def __init__(self,action_space):
    def step(self, reward, observation):
        #calculate gain
        #find gain of each flashcard
        #either rank the slate or sort
        #return best_gain
        